# 자 그러면 TfidfVectorizer를 사용한 임베딩을 입력으로 받고 파이토치로 모델로 학습해보자

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('./drive/My Drive/NLP/04. Automatic Writting Evaluation on Linear Regression (1)/codes/')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import sklearn
import torch.nn.functional as F

In [ ]:
df = pd.read_csv("AWE_1000_score.csv")

In [ ]:
df.head(2)

,Unnamed: 0,id,nationality,sex,title,content,score
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70


In [ ]:
df["sentence"] = df["content"].apply(lambda x: len(x.split(".")))
# df["content"].apply(lambda x: len(x.split("."))).value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df["content"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [ ]:
len(vectorizer.get_feature_names_out())

8093

In [ ]:
X.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
df["content_vector"]= X.todense().tolist()

In [ ]:
df.head(2)

,Unnamed: 0,id,nationality,sex,title,content,score,sentence,content_vector
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80,18,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
class MyLinearModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = torch.nn.Linear(in_features=8093, out_features= 1)

  def forward(self, x):
    return self.linear(x)


myLin = MyLinearModel()

In [ ]:
optimizer = torch.optim.SGD(myLin.parameters(), lr=0.01)

nb_epochs = 10
for epoch in range(nb_epochs + 1):
  total_loss = 0
  for idx in range(len(df)):
    x_train = np.array([df["content_vector"][idx]])
    y_train = np.array([df["score"][idx]])
    x_train = torch.from_numpy(x_train).float()
    y_train = torch.from_numpy(y_train).int()
    hypothesis = myLin(x_train)
    # cost 계산
    cost = F.mse_loss(hypothesis.to(torch.float32), y_train.to(torch.float32))
    total_loss = total_loss + cost.item()
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
  print(epoch, total_loss / len(df))

<ipython-input-12-884ca299fa21>:13: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = F.mse_loss(hypothesis.to(torch.float32), y_train.to(torch.float32))


0 474.30973144822195
1 320.5944134304338
2 296.9348496350681
3 276.45885964992505
4 258.5004629775009
5 242.60600069564603
6 228.43702090226265
7 215.728534597676
8 204.26757134986877
9 193.88004799754853
10 184.42203980013892
